In [10]:
import json
import networkx as nx
import operator
import pickle
import os, os.path
import fnmatch
import nltk

In [11]:
with open('config.json') as handle:
    config = json.load(handle)
book = config['book_name']
Parts = config['number_of_parts']
window_size = config['temporal_window_size']

In [12]:
with open('json_files/%s/date_place_list.json' % book) as handle:
    date_place_list = json.load(handle)

with open('json_files/%s/date_person_list.json' %book) as handle:
    date_person_list= json.load(handle)

In [13]:
ent_date = {'places':{},'people':{}}
for date in date_person_list:
    if date in ent_date['people']:
        ent_date['people'][date].extend(date_person_list[date])
    else:
        ent_date['people'][date] = date_person_list[date]

for date in date_place_list:
    if date in ent_date['places']:
        ent_date['places'][date].extend(date_place_list[date])
    else:
        ent_date['places'][date] = date_place_list[date]
ent_date

{'places': {'1906': ['St. Petersburg',
   'Stuttgart',
   'Poland',
   'Bonn',
   'England',
   'Brunswick',
   'Japan',
   'Trier',
   'America',
   'London',
   'France',
   'Cologne',
   'Paris',
   'South Africa',
   'Brussels',
   'Frankfort',
   'Russia',
   'Jena',
   'Odessa',
   'Munich',
   'Krakow',
   'New York',
   'Belgium',
   'Germany',
   'Moscow',
   'Hamburg',
   'Palmerston',
   'Berlin',
   'Erfurt',
   'Leipzig',
   'Sweden',
   'Elberfeld',
   'Vienna',
   'Rhineland'],
  '1848': ['St. Petersburg',
   'Iskra',
   'Poland',
   'England',
   'Japan',
   'America',
   'Warsaw',
   'France',
   'Switzerland',
   'Paris',
   'Russia',
   'Tsushima',
   'Italy',
   'Penza',
   'Belgium',
   'Germany',
   'Kazan',
   'Berlin',
   'Erfurt'],
  '1867': ['America', 'St. Petersburg', 'Russia', 'Germany', 'Reform'],
  '1901': ['St. Petersburg',
   'Empire',
   'Iskra',
   'Stuttgart',
   'Poland',
   'Hanover',
   'England',
   'Voznesensk',
   'London',
   'America',
   'Fr

In [14]:
ent_date_rev = {}
for date in ent_date['places']:
    for ent in ent_date['places'][date]:
        e = ent.lower()
        if e in ent_date_rev:
            ent_date_rev[e].append(date)
        else:
            ent_date_rev[e] = [date]
for date in ent_date['people']:
    for ent in ent_date['people'][date]:
        e = ent.lower()
        if e in ent_date_rev:
            ent_date_rev[e].append(date)
        else:
            ent_date_rev[e] = [date]
ent_date_rev

{'st. petersburg': ['1906',
  '1848',
  '1867',
  '1901',
  '1903',
  '1912',
  '1900',
  '1891',
  '1896',
  '1890',
  '1894',
  '1897'],
 'stuttgart': ['1906', '1901', '1912', '1890', '1897'],
 'poland': ['1906',
  '1848',
  '1901',
  '1903',
  '1905',
  '1912',
  '1891',
  '1896',
  '1890',
  '1897'],
 'bonn': ['1906'],
 'england': ['1906',
  '1848',
  '1901',
  '1903',
  '1871',
  '1900',
  '1896',
  '1890',
  '1897'],
 'brunswick': ['1906'],
 'japan': ['1906', '1848', '1903', '1871', '1900'],
 'trier': ['1906'],
 'america': ['1906',
  '1848',
  '1867',
  '1901',
  '1903',
  '1871',
  '1900',
  '1890',
  '1897'],
 'london': ['1906', '1901', '1903', '1900', '1897'],
 'france': ['1906',
  '1848',
  '1901',
  '1903',
  '1905',
  '1912',
  '1871',
  '1900',
  '1891',
  '1890',
  '1897'],
 'cologne': ['1906'],
 'paris': ['1906', '1848', '1901', '1903', '1912', '1871', '1900', '1890'],
 'south africa': ['1906', '1900'],
 'brussels': ['1906'],
 'frankfort': ['1906', '1903'],
 'russia': ['

In [15]:
json.dump(ent_date, open('json_files/%s/date_to_entity_dict.json' % book,'w'),indent = 4)
json.dump(ent_date_rev, open('json_files/%s/entity_to_date_dict.json' % book,'w'),indent = 4)

In [23]:
f = open('Jawaharlal_Nehru_a_Biography/chapters.txt','r')
d = {}
# chapters = [24,29,23,47,44]
chapters = [22]
part = 1
for ch in chapters:
    d[part] = {}
    for i in range(ch):
        l = f.readline().split(':')[1].split(" ",1)[1].strip()
        d[part][i+1]=l
    part+=1
    
f.close()

In [24]:
d

{1: {1: 'Birth And Parentage',
  2: 'Childhood',
  3: 'Child Marriage',
  4: 'Playing The Husband',
  5: 'At The High School',
  6: 'A Tragedy',
  7: 'Stealing And Atonement',
  8: "My Father's Death And My Double Shame",
  9: 'Glimpses of Religion',
  10: 'Preparation For England',
  11: 'Outcaste',
  12: 'In London At last',
  13: 'My Choice',
  14: 'Playing The English Gentleman',
  15: 'Changes',
  16: 'Experiments In Dietetics',
  17: 'Shyness My Shield',
  18: 'The Canker Of Untruth',
  19: 'Acquaintance With Religions',
  20: "'Nirbal Ke Bala Rama'",
  21: 'Narayan Hemchandra',
  22: 'The Great Exhibition'}}

In [25]:
json.dump(d, open('json_files/%s/chapter_names.json' % book,'w'),indent = 4)

In [26]:
with open('json_files/%s/year_mode_dict_cassigned.json' %book) as data_file:
    chap_years = json.load(data_file)
year_chap = {}
for part in chap_years.keys():
    for ch in chap_years[part].keys():
        t = (part,ch,d[int(part)][int(ch)])
        try:
            year_chap[chap_years[part][ch]].append(t)
        except:
            year_chap[chap_years[part][ch]]=[t]
            

In [27]:
year_chap

{'1912': [('1', '1', 'Birth And Parentage')],
 '1917': [('1', '2', 'Childhood')],
 '1920': [('1', '3', 'Child Marriage')],
 '1921': [('1', '4', 'Playing The Husband'), ('1', '5', 'At The High School')],
 '1923': [('1', '6', 'A Tragedy'), ('1', '7', 'Stealing And Atonement')],
 '1927': [('1', '8', "My Father's Death And My Double Shame")],
 '1929': [('1', '9', 'Glimpses of Religion')],
 '1930': [('1', '10', 'Preparation For England')],
 '1931': [('1', '11', 'Outcaste')],
 '1933': [('1', '12', 'In London At last')],
 '1936': [('1', '13', 'My Choice'),
  ('1', '14', 'Playing The English Gentleman')],
 '1937': [('1', '15', 'Changes')],
 '1940': [('1', '16', 'Experiments In Dietetics')],
 '1942': [('1', '17', 'Shyness My Shield'),
  ('1', '18', 'The Canker Of Untruth')],
 '1946': [('1', '19', 'Acquaintance With Religions'),
  ('1', '20', "'Nirbal Ke Bala Rama'"),
  ('1', '21', 'Narayan Hemchandra')],
 '1947': [('1', '22', 'The Great Exhibition')]}

In [28]:
json.dump(year_chap, open('json_files/%s/year_chapter_chaptername_dict.json' % book,'w'),indent = 4)

In [ ]:
notation_int_map = {}
location_list_map = {}
# 0 = PERSON 1 = PLACE
notation_int_map['PERSON'] = 0
notation_int_map['GPE'] = 1
dir_path = './%s/part'% book
with open('json_files/' + str(book) + '/final_chapter_entity_map_tfidf.json') as data_file:
    entity_map = json.load(data_file)
mentions = {}
for part in range(1,Parts+1):
    
    x = 1
    no_of_chapters = len(fnmatch.filter(os.listdir(dir_path + str(part) + '/'), '*.txt'))
    print('no.of_chapters:' + str(no_of_chapters))
    print('part:' + str(part))
    while x <= no_of_chapters:
        part = str(part)
        
        target_x = no_of_chapters+1
        for i in range(x,target_x):
            with open(dir_path + str(part) + '/chapter'+ str(i) + '.txt', 'r', encoding='utf-8') as content_file:
                content = content_file.read()
            i = str(i)
            # content = unicode(content, "utf-8")
            
            for sent in nltk.sent_tokenize(content):
                for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(sent))):
                    

                    entity = ' '.join(c[0] for c in chunk)
                    if entity in entity_map[str(part)][str(i)]:
                        if entity in mentions:
                            if i in mentions[entity][part]:
                                mentions[entity][part][i].append(sent)
                            else:
                                mentions[entity][part][i]=[sent]
                        else:
                            mentions[entity]={'1':{},'2':{},'3':{},'4':{},'5':{}, '6':{}, '7':{}}
                            mentions[entity][part]={i:[sent]}
        
        x = target_x

no.of_chapters:27
part:1


In [8]:
mentions['Ahmedabad']

KeyError: 'Ahmedabad'

In [9]:
json.dump(mentions, open('json_files/%s/mentions.json' % book,'w'),indent = 4)